In [2]:
import pandas as pd 
import numpy as np
from pathlib import Path
from shapely.geometry import Polygon, Point
from shapely.ops import transform
from functools import partial
import pyproj
from classLib_forFlightplans import *
import random

In [3]:
region = [105,0,110,0,110,5,105,5]
Path.cwd()

WindowsPath('e:/ATMRI/CD_R/ATCO_preferences_interface/Scenario_interface_work/python-code')

In [4]:
air=pd.read_json("../airways.json")


def filter_airways(x1,y1,x2,y2,x3,y3,x4,y4,airway):
    filtered_airways = pd.DataFrame()

    boundary_coordinates = ([x1,y1],[x2,y2],[x3,y3],[x4,y4])

    poly = Polygon(boundary_coordinates)
    for i in range(len(airway)):
        for j in range(len(airway.segments[i])):
            point1 = Point([float(airway.segments[i][j]['lat']),float(airway.segments[i][j]['lon'])])
        # point2 = Point([float(airway.segments[i][1]['lat']),float(airway.segments[i][1]['lon'])])
            if poly.contains(point1)== True :
            # and poly.contains(point2)== True
                filtered_airways=filtered_airways.append(airway.loc[i])
    filtered_airways = filtered_airways.drop_duplicates(subset=['name'])
    return filtered_airways

In [5]:
filteraws = filter_airways(0,105,0,110,5,110,5,105,air)
filteraws = filteraws.reset_index(drop=True)
filteraws.head(), len(filteraws)

(   name                                           segments
 0  G334  [{'name': 'PULAU TIOMAN VOR/DME (VPT)', 'lat':...
 1  G580  [{'name': 'SINJON DVOR/DME (SJ)', 'lat': '1.22...
 2  L504  [{'name': 'OBDOS', 'lat': '0.418', 'lon': '106...
 3  L517  [{'name': 'TERIX', 'lat': '4.256', 'lon': '109...
 4  L625  [{'name': 'AKMON', 'lat': '8.215', 'lon': '110...,
 19)

In [6]:
"""This snippet adds transformed x and y columns to the airways file"""
# a= np.array([1.19,103.06])
a=np.array([0,105])
a_ = np.array([a[1],a[0]])
# b=  np.array([1.19,104.7])
b=  np.array([0,110])
b_ = np.array([b[1],b[0]])
# c =np.array([1.76,104.7])
c =np.array([5,110])
c_ = np.array([c[1],c[0]])
# d=  np.array([1.76,103.06])
d=  np.array([5,105])
d_ = np.array([d[1],d[0]])

horizontal_len= b[1]-a[1]
vertical_len = d[0]-a[0]

dict1=[]

for i in range(len(filteraws)):
    # print(i)
    d1= []
    for j in range(len(filteraws.segments[i])):
        # print(j)
        P = np.array([float(filteraws.segments[i][j]['lon']),float(filteraws.segments[i][j]['lat'])])
        # print(P)
        # print("^^")
        # vert_d = np.linalg.norm(np.cross(b_-a_,P-a_))/np.linalg.norm(b_-a_)
        # horiz_d = np.linalg.norm(np.cross(d_-a_,P-a_))/np.linalg.norm(d_-a_)
        vert_d = P[1]-a_[1]
        horiz_d = P[0] -a_[0]
        airdict1={
                # "name":filteraws.name[i],        
                # "segments":[{"name":filteraws.segments[i][j]['name'],
                "name":filteraws.segments[i][j]['name'],
                "lat":filteraws.segments[i][j]['lat'],
                "lon":filteraws.segments[i][j]['lon'],
                "latdms":filteraws.segments[i][j]['latdms'],
                "londms":filteraws.segments[i][j]['londms'],
                "transf_x": (900/horizontal_len)*horiz_d,
                "transf_y": (900/vertical_len)*vert_d
                }
        d1.append(airdict1)
    # dict2={"segments":d1}
    dict3 ={"name":filteraws.name[i], 
            "segments":d1}
    # print(dict3)
    dict1.append(dict3)
        
print(dict1)

[{'name': 'G334', 'segments': [{'name': 'PULAU TIOMAN VOR/DME (VPT)', 'lat': '2.916', 'lon': '104.111', 'latdms': '025459N', 'londms': '1040639E', 'transf_x': -160.01999999999924, 'transf_y': 524.88}, {'name': 'KIBOL (WMFC/WSJC FIR BDRY)', 'lat': '2.875', 'lon': '104.468', 'latdms': '025229N', 'londms': '1042805E', 'transf_x': -95.75999999999937, 'transf_y': 517.5}, {'name': 'BUNTO', 'lat': '2.669', 'lon': '105.998', 'latdms': '024008N', 'londms': '1055953E', 'transf_x': 179.64000000000084, 'transf_y': 480.42}, {'name': 'KAMIN (WSJC/WBFC FIR BDRY)', 'lat': '2.578', 'lon': '108.927', 'latdms': '023442N', 'londms': '1085536E', 'transf_x': 706.8600000000013, 'transf_y': 464.03999999999996}]}, {'name': 'G580', 'segments': [{'name': 'SINJON DVOR/DME (SJ)', 'lat': '1.222', 'lon': '103.856', 'latdms': '011319N', 'londms': '1035120E', 'transf_x': -205.92000000000098, 'transf_y': 219.96}, {'name': 'HOSBA (R079/34 DME SJ)(R103/24 DME VTK)', 'lat': '1.330', 'lon': '104.405', 'latdms': '011948N', 

In [7]:
import json
with open('scenario_data/airwaysforFlightPlan.json', 'w') as json_file:
    json.dump(dict1, json_file)

In [15]:
df=pd.read_json('scenario_data/airwaysforFlightPlan.json')
waypointList = []
for i in range(len(df)):
    for j in range(len(df.segments[i])):
        waypointList.append(WayPoint(df.segments[i][j]['name'],[df.segments[i][j]['transf_x'],900-df.segments[i][j]['transf_y']]))
        
len(waypointList),len(df.segments)

(109, 19)

In [9]:
for a in range(len(df.segments)):
    print(a,len(df.segments[a]))

0 4
1 6
2 2
3 2
4 4
5 5
6 6
7 10
8 4
9 5
10 9
11 8
12 5
13 8
14 3
15 11
16 7
17 8
18 2


In [10]:
airwayList=[]
a=0
for i in range(len(df.segments)):
    x=len(df.segments[i])
    # print(x)
    for j in range(a,len(waypointList[:a+x])-1):
        li=[waypointList[j],waypointList[j+1]]
        # print(li)
        airwayList.append(Airways('aw'+ str(j),li[0],li[1]))
    a+=x

In [11]:
# ac_per_scenario = 8
# aircraftList=[]
# aircraft_name=[]
# for n in range(1,ac_per_scenario+1):
#     aircraft_name.append('ac'+str(n))
# for i in range(ac_per_scenario):
#     point=np.random.choice(airwayList)
#     d=[20,30,40,50,60,80,100,120,140,150,170,190,210,220,230,240,250] #offset,
#     aircraft = Aircraft(str(aircraft_name[i]), point , np.random.choice(d)) #this will create multiple scenarios
#     # aircraftList.append(aircraft)

In [12]:
""""here, the list aircraft_paths contains the waypoint lists that an aircraft will move on"""
aircraft_paths=[] 
b=0
for i in range(len(df.segments)):
    x= len(df.segments[i])
    segment=[]
    for j in range(b,len(waypointList[:b+x])):
        li=waypointList[j]
        segment.append(li)
    aircraft_paths.append(segment)
    b+=x

In [59]:
for i in range(len(aircraft_paths[7])):
    print(aircraft_paths[7][i])

waypoint class:DUDIS (WSJC/VVTS FIR BDRY)  x co-ord: 325.8000000000004  y co-ord: -360.0
waypoint class:MABLI  x co-ord: 218.3399999999989  y co-ord: 128.15999999999997
waypoint class:OPULA  x co-ord: 243.90000000000072  y co-ord: 264.24
waypoint class:ONAPO  x co-ord: 249.84000000000094  y co-ord: 296.28
waypoint class:OMLIV  x co-ord: 264.5999999999998  y co-ord: 374.4
waypoint class:OMBAP  x co-ord: 278.1000000000003  y co-ord: 446.22
waypoint class:OLSAM  x co-ord: 295.2000000000001  y co-ord: 537.12
waypoint class:OBLOT  x co-ord: 305.27999999999963  y co-ord: 591.12
waypoint class:OBGET  x co-ord: 316.62000000000006  y co-ord: 650.7
waypoint class:KIKOR (WSJC/WIIZ FIR BDRY)  x co-ord: 376.2000000000006  y co-ord: 968.22


In [16]:
for i in range(len(aircraft_paths)):
    print(len(aircraft_paths[i]))

4
6
2
2
4
5
6
10
4
5
9
8
5
8
3
11
7
8
2


In [ ]:
ed=[1,2,3,4]
aircraft_paths[0][]

In [12]:
# all_airways_combined=[]
# for i in range(len(df.segments[i])): # segments means number of wps in that segment
#     segmentlength = len(df.segments[i])
#     c=0
#     airways_combined =[]
#     for j in range(len(airwayList[:c+segmentlength-1])):
#         airways_combined.append(airwayList[j])
#     all_airways_combined.append(airways_combined)



In [22]:
aircraft_paths_x = []
aircraft_paths_y = []
path_waypoints   = []
for i in range(len(aircraft_paths)):
    x_list=[]
    y_list=[]
    wps_list=[]
    for j in range(len((aircraft_paths[i]))):
        wps_list.append(aircraft_paths[i][j].name)
        x=aircraft_paths[i][j].x
        y=aircraft_paths[i][j].y
        x_list.append(x)
        y_list.append(y)
    aircraft_paths_x.append(x_list)
    aircraft_paths_y.append(y_list)
    path_waypoints.append(wps_list)

In [23]:
aircraft_paths_x[2]

[347.57999999999953, 720.0]

In [24]:
# startEnd_point=[]
# for i in range(len(aircraft_paths)):
#     point=[aircraft_paths[i][0],aircraft_paths[i][-1]]
#     startEnd_point.append(point)

In [39]:
ac_pathlongerthan2= []
for i in range(len(aircraft_paths)):
    if len(aircraft_paths[i])>2:
        ac_pathlongerthan2.append(aircraft_paths[i])
len(ac_pathlongerthan2)

16

In [25]:
aircraft_type=['medium','heavy','super_heavy']

In [40]:
ac_per_scenario = 8 
aircraftList=[]
aircraft_name=[]
for n in range(1,ac_per_scenario+1):
    aircraft_name.append('ac'+str(n))
for i in range(ac_per_scenario):
    route = np.random.choice(ac_pathlongerthan2)
    # print(point)
    d=[20,30,40,50,60,80,100,120,140,150,170,190,210,220] #offset,
    aircraft = Aircraft(str(aircraft_name[i]), route[0] ,route[-1], route,  np.random.choice(d)) #this will create multiple scenarios
    aircraftList.append(aircraft)


In [41]:
      
x_dirlist=[]
y_dirlist=[]
for i in range(len(aircraftList)):
    x_dir =[]
    y_dir =[]
    for j in range(len(aircraftList[i].route)-1):
        startpoint = np.array([aircraftList[i].route[j].x, aircraftList[i].route[j].y])
        endpoint = np.array([aircraftList[i].route[j+1].x, aircraftList[i].route[j+1].y])
        segment = np.linalg.norm(startpoint - endpoint)
        xdir = (endpoint[0] - startpoint[0])/segment
        ydir = (endpoint[1] - startpoint[1])/segment
        x_dir.append(xdir)
        y_dir.append(ydir)
    x_dirlist.append(x_dir)
    y_dirlist.append(y_dir)

In [42]:
len((x_dirlist))

# (aircraftList[0].route[0].x)

8

In [43]:
vel=0.25
timelist=[]
x_loclist=[]
y_loclist=[]
total_time=[]
for i in range(len(aircraftList)):
    x_loc=[]
    y_loc=[]
    time=[]
    section_time=[]
    for j in range(len(aircraftList[i].route)-1):
        startpoint = np.array([aircraftList[i].route[j].x, aircraftList[i].route[j].y])
        endpoint = np.array([aircraftList[i].route[j+1].x, aircraftList[i].route[j+1].y])
        distance = np.linalg.norm(startpoint - endpoint)
        totaltime = distance/vel    #(time per kilometer in seconds, bwtween two waypoints)
        
        for t in range(int(totaltime)):
            x_point = startpoint[0] + t*x_dirlist[i][j]*vel
            y_point = startpoint[1] + t*y_dirlist[i][j]*vel
            x_loc.append(x_point)
            y_loc.append(y_point)
            time.append(t)
            
        section_time.append(totaltime)
    print(len(section_time))
    timelist.append(time)
    x_loclist.append(x_loc)
    y_loclist.append(y_loc)
    total_time.append(section_time)

5
3
8
6
3
2
7
3


In [44]:
(total_time[0][0])+ (total_time[0][1])+ (total_time[0][2])

1575.8544154057204

In [45]:
all_dirs_x=[]
all_dirs_y=[]
for i in range(len(total_time)):
    x_list=[]
    y_list=[]
    for j in range(len(total_time[i])):
        x = np.repeat(x_dirlist[i][j], int(total_time[i][j]))
        y = np.repeat(y_dirlist[i], int(total_time[i][j]))
        print(x)
        x= x.tolist()
        y= y.tolist()
        # print(len(x))
        # print(len(y[]))
        x_list.append(x)
        y_list.append(y)
    
    all_dirs_x.append(x_list)
    all_dirs_y.append(y_list)


0.96368407 0.96368407 0.96368407 0.96368407 0.96368407
 0.96368407 0.96368407 0.96368407 0.96368407 0.96368407 0.96368407
 0.96368407 0.96368407 0.96368407 0.96368407 0.96368407 0.96368407
 0.96368407 0.96368407 0.96368407 0.96368407 0.96368407 0.96368407
 0.96368407 0.96368407 0.96368407 0.96368407 0.96368407 0.96368407
 0.96368407 0.96368407 0.96368407 0.96368407 0.96368407 0.96368407
 0.96368407 0.96368407 0.96368407 0.96368407 0.96368407 0.96368407
 0.96368407 0.96368407 0.96368407 0.96368407 0.96368407 0.96368407
 0.96368407 0.96368407 0.96368407 0.96368407 0.96368407 0.96368407
 0.96368407 0.96368407 0.96368407 0.96368407 0.96368407 0.96368407
 0.96368407 0.96368407 0.96368407 0.96368407 0.96368407 0.96368407
 0.96368407 0.96368407 0.96368407 0.96368407 0.96368407 0.96368407
 0.96368407 0.96368407 0.96368407 0.96368407 0.96368407 0.96368407
 0.96368407 0.96368407 0.96368407 0.96368407 0.96368407 0.96368407
 0.96368407 0.96368407 0.96368407 0.96368407 0.96368407 0.96368407
 0.9636

In [46]:
len(all_dirs_x[0])

5

In [47]:
import operator
from functools import reduce
x_directions_flat=[]
y_directions_flat=[]
for i in range(len(x_dirlist)):
    x_l= all_dirs_x[i]
    x_flat = reduce(operator.concat,x_l)
    x_directions_flat.append(x_flat)

    y_l = all_dirs_y[i]
    y_flat = reduce(operator.concat,y_l)
    y_directions_flat.append(y_flat)

In [48]:
dict4={}
for i in range(len(waypointList)):
    waydict={waypointList[i].name:{
        'x':waypointList[i].x,
        'y':waypointList[i].y}}
    dict4.update(waydict)


dict5={}
for i in range(len(airwayList)):
    airwaydict= {airwayList[i].name:{
        'start':airwayList[i].start_wp,
        'end' : airwayList[i].end_wp}}
    dict5.update(airwaydict)


dict6={}
for i in range(len(aircraftList)):
    aircraftdict= {
        aircraftList[i].name:{
        'x': aircraftList[i].start_wp_x,
        'y':aircraftList[i].start_wp_y,
        'dir_x': x_dirlist[i][0],
        'dir_y': y_dirlist[i][0],
        'x_dirs':x_directions_flat[i],
        'y_dirs':y_directions_flat[i],
        'type': np.random.choice(aircraft_type),
        'altitude':360,
        'path_waypoints': path_waypoints[i],
        'x_path' : x_loclist[i],
        'y_path' : y_loclist[i],
        'timestep': timelist[i]
        }
        }
    dict6.update(aircraftdict)

In [49]:
df2=pd.read_json('SgSectors_LonLat.json')


"""the sqaure region with 'a' as the bottom left point (moving anticlock abcd)
    a,b,c,d= [lon,lat] format"""
a = np.array([102,1])
b = np.array([108,1])
c = np.array([108,7])
d = np.array([102,7])

horizontal_length  = b[0]-a[0]
vertical_length = d[1]-a[1]
keylist=['sector1','sector2','sector3','sector4','sector5','sector6','sector7','sector8']
l1=[[],[],[],[],[],[],[],[]]
for i in range(len(keylist)):
    for j in range(len(df2.Sectors[0][keylist[i]])):
        P= np.array([df2.Sectors[0][keylist[i]][j][0],df2.Sectors[0][keylist[i]][j][1]])
        # print(P)
        vert_d = P[1]-a[1]
        horiz_d= P[0]-a[0]
        l1[i].append([(900/horizontal_length)*horiz_d,900-(900/vertical_length)*vert_d])



data_dict={'waypoints':dict4, 'airways':dict5,'aircrafts':dict6, 
        'sectors':{
                    'sector1':{'type': 'secondary', 'boundary': l1[0]},
                    'sector2':{'type': 'secondary', 'boundary': l1[1]},
                    'sector3':{'type': 'secondary', 'boundary': l1[2]},
                    'sector4':{'type': 'secondary', 'boundary': l1[3]},
                    'sector5':{'type': 'secondary',  'boundary': l1[4]},
                    'sector6':{'type': 'primary', 'boundary':  l1[5]},
                    'sector7':{'type':'secondary', 'boundary': l1[6]},
                    'sector8':{'type':'secondary' ,'boundary':l1[7]}
                    }}

In [50]:
Path.cwd()

WindowsPath('e:/ATMRI/CD_R/ATCO_preferences_interface/Scenario_interface_work/python-code')

In [51]:
datalist=[]
datalist.append(data_dict)
import json
with open('scenario_data/12August_FlightPlanTest_1.json', 'w') as json_file:
    json.dump(datalist, json_file)